In [23]:
import numpy as np

In [9]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("jamiewelsh2/rap-lyrics")

print("Path to dataset files:", path)


# Load the dataset
df = pd.read_csv(path + "/updated_rappers.csv")
display(df.head())



Path to dataset files: C:\Users\Nima\.cache\kagglehub\datasets\jamiewelsh2\rap-lyrics\versions\1


,Unnamed: 0,artist,song,lyric,next lyric
0,0,Fetty Wap,Trap Queen,rgf productions,remy boyz yahah
1,0,Fetty Wap,Trap Queen,remy boyz yahah,1738 ayy
2,0,Fetty Wap,Trap Queen,1738 ayy,im like hey whats up hello
3,0,Fetty Wap,Trap Queen,im like hey whats up hello,seen yo pretty ass soon as you came in the door
4,0,Fetty Wap,Trap Queen,seen yo pretty ass soon as you came in the door,i just wanna chill got a sack for us to roll


In [10]:


# Load data
def load_data(path):
    df = pd.read_csv(path + "/updated_rappers.csv")
    df = df[['lyric', 'next lyric']].dropna()
    return df

df = load_data(path)
display(df.head())



,lyric,next lyric
0,rgf productions,remy boyz yahah
1,remy boyz yahah,1738 ayy
2,1738 ayy,im like hey whats up hello
3,im like hey whats up hello,seen yo pretty ass soon as you came in the door
4,seen yo pretty ass soon as you came in the door,i just wanna chill got a sack for us to roll


In [11]:
# Preprocess data
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Special characters weghalen
    text = re.sub(r'\s+', ' ', text).strip()  # Extra whitespace weghalen
    return text


In [12]:
# toepassen van clean_text op de kolommen lyric en next lyric
def preprocess_lyrics(df):
    df['lyric'] = df['lyric'].apply(clean_text)
    df['next lyric'] = df['next lyric'].apply(clean_text)
    return df



In [ ]:
# Tokenize data
from collections import Counter
def build_vocab(df, vocab_size=5000):
    words = ' '.join(df['lyric']).split()
    word_counts = Counter(words)
    most_common = word_counts.most_common(vocab_size - 1)
    vocab = {word: idx+1 for idx, (word, _) in enumerate(most_common)}  
    vocab['<UNK>'] = vocab_size  # Unknown words
    return vocab


def text_to_sequence(text, vocab):
    return [vocab.get(word, vocab['<UNK>']) for word in text.split()]

# How do I make train and test set the same size using bag of words. (n.d.).
# Stack Overflow. https://stackoverflow.com/questions/67494914/how-do-i-make-train-and-test-set-the-same-size-using-bag-of-words

In [ ]:
# Data naar numerieke waarden omzetten
def prepare_data(filepath, vocab_size=5000):
    df = load_data(filepath)
    df = preprocess_lyrics(df)
    vocab = build_vocab(df, vocab_size)
    df['lyric_seq'] = df['lyric'].apply(lambda x: text_to_sequence(x, vocab))
    df['next_lyric_seq'] = df['next lyric'].apply(lambda x: text_to_sequence(x, vocab))
    return df, vocab

In [17]:
df, vocab = prepare_data(path)

In [20]:
display(df.head())

,lyric,next lyric,lyric_seq,next_lyric_seq
0,rgf productions,remy boyz yahah,"[5000, 5000]","[2462, 3022, 5000]"
1,remy boyz yahah,ayy,"[2462, 3022, 5000]",[321]
2,ayy,im like hey whats up hello,[321],"[11, 14, 227, 193, 18, 929]"
3,im like hey whats up hello,seen yo pretty ass soon as you came in the door,"[11, 14, 227, 193, 18, 929]","[254, 98, 568, 124, 622, 84, 3, 199, 9, 1, 418]"
4,seen yo pretty ass soon as you came in the door,i just wanna chill got a sack for us to roll,"[254, 98, 568, 124, 622, 84, 3, 199, 9, 1, 418]","[2, 36, 72, 757, 21, 4, 1583, 19, 122, 6, 255]"


Hier is een uitgebreide, gestructureerde uitleg van de **Mamba-class**, inclusief duidelijke referenties naar de paper. Je kunt dit direct in je Markdown-document plaatsen.  

---

## **Implementatie en uitleg van het Mamba-model**  

De **Mamba-architectuur** is een nieuw type sequentiemodel dat gebruik maakt van **State Space Models (SSMs)**, in plaats van zelf-attentie zoals in Transformers. In tegenstelling tot klassieke SSMs, die een vast geheugen hebben, introduceert Mamba een **selectief mechanisme** dat het mogelijk maakt om irrelevante informatie te filteren en belangrijke tokens vast te houden. Dit maakt het model efficiënter en beter in het verwerken van lange sequenties.  

Onze implementatie is gebaseerd op de principes uit de paper *Mamba: Linear-Time Sequence Modeling with Selective State Spaces* (Gu & Dao, 2024) en bouwt stap voor stap een eenvoudige versie van het model op.  

---

### **1. De Basis: State Space Models (SSMs)**  

In de paper wordt uitgelegd dat **State Space Models (SSMs)** werken met een verborgen toestand (\( h_t \)) die bij elke nieuwe token wordt bijgewerkt volgens de volgende formule:  

\[
h_t = A h_{t-1} + B x_t
\]

en de uiteindelijke output wordt berekend als:

\[
y_t = C h_t
\]

Dit betekent dat de toestand van het model op elk moment afhangt van de vorige toestand (\( h_{t-1} \)) en de huidige input (\( x_t \)). Dit concept komt uit de klassieke **lineaire recursieve netwerken** en wordt gebruikt om lange-afstandsafhankelijkheden in tekst te modelleren.  

**Referentie:** Zie sectie *State Space Models* (pagina 3) waar deze basisformules worden geïntroduceerd:  

*"S4 models are defined with four parameters (Δ, A, B, C), which define a sequence-to-sequence transformation in two stages."*  

Om dit in onze code om te zetten, gebruiken we een eenvoudige implementatie waarbij een verborgen toestand (\( h_t \)) wordt bijgehouden en geüpdatet bij elke nieuwe token:  

```python
self.hidden_state = np.tanh(np.dot(self.W, self.hidden_state) + np.dot(self.U, x) + self.b)
```

Hierbij:
- **\( W \)** is de matrix die de toestand bijwerkt op basis van de vorige toestand.
- **\( U \)** transformeert de invoer naar een vector die kan worden toegevoegd aan de verborgen toestand.
- **\( b \)** is een bias-term.
- **\( \tanh \)** zorgt ervoor dat de waarden binnen een bepaald bereik blijven en voorkomt dat de toestand explodeert.  

---

### **2. Selectieve Informatieopslag: De Verbetering van Mamba**  

Een van de kernproblemen van eerdere SSMs was dat ze **tijd-invariant** waren: de overgangsmatrix (\( A \)) veranderde niet afhankelijk van de invoer. Dit betekende dat ze **geen content-afhankelijke beslissingen konden nemen**, zoals wanneer een woord belangrijk is om te onthouden.  

Mamba lost dit op door de **SSM-parameters input-afhankelijk te maken**. Dit betekent dat de manier waarop het model informatie doorgeeft, varieert afhankelijk van de invoer die het krijgt. Hierdoor kan Mamba:  
- Relevante tokens opslaan en irrelevante negeren.  
- Informatie veel efficiënter doorgeven.  

**Referentie:** Zie sectie *Selection Mechanism* (pagina 5), waar wordt uitgelegd hoe Mamba een selectiemechanisme toevoegt aan de klassieke SSM-structuur:  

*"Building on intuition based on important synthetic tasks such as selective copy and induction heads, we design a simple selection mechanism by parameterizing the SSM parameters based on the input."*  

Onze implementatie vertaalt dit concept als volgt:  

```python
for token in input_seq:
    x = np.zeros(self.vocab_size)
    x[token] = 1  # One-hot encoding van de input
    self.hidden_state = np.tanh(np.dot(self.W, self.hidden_state) + np.dot(self.U, x) + self.b)
```

Hier zorgt de **one-hot encoding** ervoor dat elk woord wordt omgezet in een vector, en de overgangsformule past zich dynamisch aan op basis van deze invoer.

---

### **3. Autoregressieve Generatie**  

Omdat Mamba ontworpen is als een **autoregressief model**, kan het worden gebruikt om tekst te genereren. Dit betekent dat we het model een beginzin geven en het vervolgens **stap voor stap nieuwe tokens laat voorspellen** op basis van de verborgen toestand.  

De paper beschrijft dat Mamba **geen cache van eerdere tokens nodig heeft**, in tegenstelling tot Transformers. Dit maakt het model veel efficiënter in inference.  

**Referentie:** Zie sectie *Inference and Scaling* (pagina 2):  

*"Unrolling the model autoregressively during inference requires only constant time per step since it does not require a cache of previous elements."*  

In onze implementatie gebeurt dat zo:  

```python
def generate(self, start_seq, length=10):
    generated = list(start_seq)
    for _ in range(length):
        state = self.forward(generated[-1:])
        next_token = np.argmax(state)  # Kies de meest waarschijnlijke volgende token
        generated.append(next_token)
    return generated
```

Hierbij:
- **We starten met een beginzin** (`start_seq`).
- **Elke nieuwe token wordt gegenereerd door het model** op basis van de vorige token.
- **De output wordt gegenereerd met `np.argmax(state)`**, wat de meest waarschijnlijke token selecteert.

Dit betekent dat als we bijvoorbeeld de input `"ayy"` geven, het model mogelijk `"im like hey whats up"` genereert als de volgende woorden.








In [21]:
# Mamba model implementeren
class Mamba:
    def __init__(self, vocab_size, hidden_size=128):
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.W = np.random.randn(hidden_size, hidden_size) * 0.1
        self.U = np.random.randn(hidden_size, vocab_size) * 0.1
        self.b = np.zeros(hidden_size)
        self.hidden_state = np.zeros(hidden_size)
    
    def forward(self, input_seq):
        for token in input_seq:
            x = np.zeros(self.vocab_size)
            x[token] = 1  # One-hot encoding
            self.hidden_state = np.tanh(np.dot(self.W, self.hidden_state) + np.dot(self.U, x) + self.b)
        return self.hidden_state
    
    def generate(self, start_seq, length=10):
        generated = list(start_seq)
        for _ in range(length):
            state = self.forward(generated[-1:])
            next_token = np.argmax(state)  # Simplified selection
            generated.append(next_token)
        return generated
    


In [27]:
# example usage
model = Mamba(vocab_size=len(vocab))
generated_seq = model.generate([vocab['ayy']], length=30)

# Convert sequence back to text
generated_text = ' '.join([list(vocab.keys())[idx] for idx in generated_seq])
print(generated_text)


free now ill if em not my gon in we are know what have be girl them what us or got thats her cant said on my gon yeah life where
